In [2]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import numpy as np
import pypyodbc
import csv
import os
import tempfile
import statsmodels.formula.api as smf
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
puma_ref = pd.read_excel('http://www.nyc.gov/html/dcp/download/census/nyc2010census_tabulation_equiv.xlsx',header=3)
# alternatively, use shapefile from https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty
puma_list = puma_ref['PUMA'].unique()[~np.isnan(puma_ref['PUMA'].unique())]

In [4]:
pums_data = pd.read_csv('ss14hny.csv')
pums_data['PUMA'] = pums_data[['PUMA00','PUMA10']].max(axis=1)

In [5]:
nyc_pums = pd.DataFrame()
for puma in puma_list:
    nyc_pums = nyc_pums.append(pums_data.query('PUMA == @puma'))
    print nyc_pums.shape[0],'rows after addition of PUMA',str(puma)

2532 rows after addition of PUMA 3704.0
4290 rows after addition of PUMA 3706.0
7271 rows after addition of PUMA 3705.0
9141 rows after addition of PUMA 3703.0
11081 rows after addition of PUMA 3708.0
13603 rows after addition of PUMA 3702.0
15801 rows after addition of PUMA 3707.0
18926 rows after addition of PUMA 3710.0
21257 rows after addition of PUMA 3701.0
24034 rows after addition of PUMA 3709.0
27707 rows after addition of PUMA 4017.0
30805 rows after addition of PUMA 4013.0
33408 rows after addition of PUMA 4003.0
36267 rows after addition of PUMA 4014.0
38410 rows after addition of PUMA 4018.0
41972 rows after addition of PUMA 4004.0
44269 rows after addition of PUMA 4007.0
46446 rows after addition of PUMA 4002.0
50660 rows after addition of PUMA 4009.0
53223 rows after addition of PUMA 4005.0
55807 rows after addition of PUMA 4006.0
57735 rows after addition of PUMA 4011.0
60478 rows after addition of PUMA 4008.0
63099 rows after addition of PUMA 4010.0
66109 rows after add

In [6]:
AMI_60_max = {1:36300,2:41460,3:46620,4:51780,5:56940,6:62100,7:67260,8:72420}
# 80 pct figures for 2015 from http://www1.nyc.gov/assets/hpd/downloads/pdf/NSP-tenant-lease-guidelines.pdf
AMI_80_max = {1:48400,2:55280,3:62160,4:69040,5:74640,6:80160,7:85680,8:91200}
nyc_pums['AMI_60'] = 0
for n in AMI_60_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums['AMI_80'] = 0
for n in AMI_80_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1

In [7]:
age_grouping = {1:'PreWar',2:'PostWar',3:'PostWar',4:'PostWar',5:'PostWar',6:'PostWar',7:'PostWar',8:'Modern'}
nyc_pums['AgeGroup'] = ''
for n in age_grouping.items():
    nyc_pums.loc[nyc_pums['YBL']==n[0],'AgeGroup'] = n[1]
nyc_pums.loc[nyc_pums['YBL']>n[0],'AgeGroup'] = n[1]

In [8]:
# size_grouping = {1:'Small',2:'Small',3:'Small',4:'Small',5:'Medium',6:'Medium',7:'Medium',8:'Large',9:'Large'}
size_grouping = {1:'Small',2:'Small',3:'Small',4:'Small',5:'Small',6:'Large',7:'Large',8:'Large',9:'Large'}
nyc_pums['SizeGroup'] = ''
for n in size_grouping.items():
    nyc_pums.loc[nyc_pums['BLD']==n[0],'SizeGroup'] = n[1]
nyc_pums.loc[nyc_pums['BLD']>n[0],'SizeGroup'] = n[1]

In [9]:
nyc_pums['ResidencyGroup'] = ''
nyc_pums.loc[nyc_pums['MV']<=4,'ResidencyGroup'] = 'Under10'
nyc_pums.loc[nyc_pums['MV']==5,'ResidencyGroup'] = '10Years'
nyc_pums.loc[nyc_pums['MV']==6,'ResidencyGroup'] = '20Years'
nyc_pums.loc[nyc_pums['MV']==7,'ResidencyGroup'] = '30Years'

In [10]:
print 'Median HH income in all NYC PUMAs:', nyc_pums['HINCP'].median()
print 'Median HH income in all NYC PUMAs, 2-person HH:', nyc_pums.query('NP == 2')['HINCP'].median()

Median HH income in all NYC PUMAs: 52000.0
Median HH income in all NYC PUMAs, 2-person HH: 59600.0


In [11]:
print 'Example median HH income in PUMA of 945 Aldus St, Bronx:', nyc_pums.query('NP == 2 & PUMA == 3710')['HINCP'].median()
print('Example median HH income in PUMA of 109-20 Queens Blvd, Queens:'), nyc_pums.query('NP == 2 & PUMA == 4108')['HINCP'].median()
print('Example median HH income in PUMA of 16 S Portland Ave, Brooklyn:'), nyc_pums.query('NP == 2 & PUMA == 4004')['HINCP'].median()

Example median HH income in PUMA of 945 Aldus St, Bronx: 22000.0
Example median HH income in PUMA of 109-20 Queens Blvd, Queens: 72150.0
Example median HH income in PUMA of 16 S Portland Ave, Brooklyn: 100000.0


In [12]:
# excluding group quarters and vacant household records
nyc_pums = nyc_pums.query('TYPE == 1 & BLD > 1 & BLD < 10')
nyc_pums = nyc_pums[nyc_pums['VACS'].isnull()]

In [13]:
print 'Percentage of households who own that are considered Low Income:', nyc_pums.query('TEN < 3')['AMI_80'].mean()
nyc_pums.query('TEN < 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who own that are considered Low Income: 0.364142139268


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.217252,0.360374,0.240803
Small,0.323671,0.382688,0.389072


In [14]:
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who rent that are considered Low Income: 0.36415481864


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.530627,0.677205,0.651855
Small,0.661379,0.638255,0.626999


In [15]:
print 'Also add length of residency category...'
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns=['ResidencyGroup','AgeGroup'],values='AMI_80',aggfunc='mean')

Also add length of residency category...
Percentage of households who rent that are considered Low Income: 0.650481622307


ResidencyGroup   10Years                       20Years             30Years  \
AgeGroup          Modern   PostWar    PreWar   PostWar    PreWar   PostWar   
SizeGroup                                                                    
Large           0.802469  0.749725  0.725532  0.748691  0.742024  0.761013   
Small           0.756098  0.644608  0.671696  0.669887  0.666667  0.687140   

ResidencyGroup             Under10                      
AgeGroup          PreWar    Modern   PostWar    PreWar  
SizeGroup                                               
Large           0.732872  0.519484  0.610500  0.590330  
Small           0.707792  0.656390  0.631222  0.604679

In [16]:
# can we identify renters who live in condo buildings?
nyc_pums.query('TEN == 3 and FCONP == 1')
# it appears not

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP77,WGTP78,WGTP79,WGTP80,PUMA,AMI_60,AMI_80,AgeGroup,SizeGroup,ResidencyGroup


In [17]:
# dependents to test: HINCP, RNTP, GRNTP, NPF, TAXP
# independents to test: YBL (ord), TEN (c), BDSP, BLD, VACS - perhaps as a filter?
# also of interest: MV

In [18]:
# predict rent paid for renters
# mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [19]:
# predict income for renters
# mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [20]:
# predict income for owners
# mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
# print(mod.summary())

In [21]:
# predict income for owners with property tax as independent, but without bedrooms or size of building (covairance)
# mod = smf.ols(formula='HINCP ~ TAXP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
# print(mod.summary())

In [22]:
# predict rent paid for renters with grouped building ages
# mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(AgeGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [23]:
# predict income for renters with grouped building ages/sizes and length of residence
mod = smf.ols(formula='HINCP ~ NP + C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(ResidencyGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     272.7
Date:                Thu, 10 Mar 2016   Prob (F-statistic):               0.00
Time:                        12:38:01   Log-Likelihood:            -9.9449e+05
No. Observations:               78900   AIC:                         1.989e+06
Df Residuals:                   78838   BIC:                         1.990e+06
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
Intercept       

In [55]:
# logit_mod = smf.logit(formula='AMI_80 ~ C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(ResidencyGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
logit_mod = smf.logit(formula='AMI_80 ~ C(SizeGroup) + C(AgeGroup) + C(PUMA)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.586582
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 AMI_80   No. Observations:                78900
Model:                          Logit   Df Residuals:                    78842
Method:                           MLE   Df Model:                           57
Date:                Thu, 10 Mar 2016   Pseudo R-squ.:                 0.09359
Time:                        14:06:48   Log-Likelihood:                -46281.
converged:                       True   LL-Null:                       -51060.
                                        LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                  0.1508      0.069      2.195      0.028         0.016    

In [56]:
mfx = logit_mod.get_margeff()
print mfx.summary()

        Logit Marginal Effects       
Dep. Variable:                 AMI_80
Method:                          dydx
At:                           overall
                            dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
C(SizeGroup)[T.Small]     -0.0641      0.004    -16.347      0.000        -0.072    -0.056
C(AgeGroup)[T.PostWar]     0.0765      0.006     12.179      0.000         0.064     0.089
C(AgeGroup)[T.PreWar]      0.0675      0.006     10.537      0.000         0.055     0.080
C(PUMA)[T.3702]            0.1572      0.018      8.641      0.000         0.122     0.193
C(PUMA)[T.3703]            0.0309      0.020      1.538      0.124        -0.008     0.070
C(PUMA)[T.3704]            0.1022      0.017      5.862      0.000         0.068     0.136
C(PUMA)[T.3705]            0.3210      0.018     17.452      0.000         0.285     0.357
C(PUMA)[T.3706]            0.

In [57]:
def clean_params(params):
    new_index = []
    for i in params.index:
        if i[:2]=='C(':
            l = i.find('[T.') + 3
            r = i.find(']')
            new_index.append(i[l:r])
        else:
            new_index.append(i)
    return new_index

In [58]:
logit_params = pd.Series(data=logit_mod.params.values,index=clean_params(logit_mod.params))

In [59]:
logit_params.to_pickle('LOGIT_PARAM')